In [1]:
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime

In [2]:
import mysql.connector
from dotenv import load_dotenv
import os

In [3]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'JPM', 'BAC']
start_date = "2014-01-01"
end_date = "2024-01-31"

In [4]:
df = yf.download(tickers, start_date, end_date, auto_adjust=True)
df

[*********************100%***********************]  10 of 10 completed


Price            Close                                                 \
Ticker            AAPL        AMZN        BAC       GOOGL         JPM   
Date                                                                    
2014-01-02   17.234297   19.898500  13.049142   27.787050   43.285351   
2014-01-03   16.855730   19.822001  13.300396   27.584349   43.619980   
2014-01-06   16.947643   19.681499  13.503024   27.891895   43.872795   
2014-01-07   16.826445   19.901501  13.373345   28.429604   43.367146   
2014-01-08   16.933002   20.096001  13.438183   28.488768   43.776127   
...                ...         ...        ...         ...         ...   
2024-01-24  193.551773  156.869995  32.340477  148.332703  167.550354   
2024-01-25  193.223404  157.750000  32.742527  151.494858  169.948166   
2024-01-26  191.481918  159.119995  32.781750  151.814087  169.299561   
2024-01-29  190.795303  161.259995  32.958260  153.130814  169.741791   
2024-01-30  187.123260  159.000000  34.115383  151.085892  173.220535   

Price                                                                  ...  \
Ticker            META        MSFT        NFLX       NVDA        TSLA  ...   
Date                                                                   ...   
2014-01-02   54.545769   31.058563   51.831429   0.373966   10.006667  ...   
2014-01-03   54.396221   30.849613   51.871429   0.369486    9.970667  ...   
2014-01-06   57.028294   30.197687   51.367142   0.374438    9.800000  ...   
2014-01-07   57.746132   30.431700   48.500000   0.380568    9.957333  ...   
2014-01-08   58.055199   29.888443   48.712856   0.385756   10.085333  ...   
...                ...         ...         ...        ...         ...  ...   
2024-01-24  389.527161  399.572021  544.869995  61.344604  207.830002  ...   
2024-01-25  391.999725  401.864899  562.000000  61.599533  182.630005  ...   
2024-01-26  392.956879  400.931885  570.419983  61.013702  183.250000  ...   
2024-01-29  399.816193  406.678925  575.789978  62.447296  190.929993  ...   
2024-01-30  398.859100  405.557281  562.849976  62.756203  191.589996  ...   

Price          Volume                                                      \
Ticker           AAPL      AMZN        BAC      GOOGL       JPM      META   
Date                                                                        
2014-01-02  234684800  42756000  148709900   72783144  15627600  43195500   
2014-01-03  392467600  44204000  129921800   66601332  14214100  38246200   
2014-01-06  412610800  63412000  114431300   70701228  17550700  68852600   
2014-01-07  317209200  38320000  110605100  102001896  17851200  77207400   
2014-01-08  258529600  46330000  101036400   89610300  14687400  56682400   
...               ...       ...        ...        ...       ...       ...   
2024-01-24   53631300  48547300   36403900   25233500   9967100  15698500   
2024-01-25   54822100  43638600   46204000   29149100   8873500  15091100   
2024-01-26   44594000  51047400   30839800   26115500   7443000  13163700   
2024-01-29   47145600  45270400   30269300   27784300   6971200  18742400   
2024-01-30   55859400  45207400   60315700   36331800  10822100  18614700   

Price                                                 
Ticker          MSFT      NFLX       NVDA       TSLA  
Date                                                  
2014-01-02  30632200  12325600  260092000   92826000  
2014-01-03  31134800  10817100  259332000   70425000  
2014-01-06  43603700  15501500  409492000   80416500  
2014-01-07  35802800  36167600  333288000   75511500  
2014-01-08  59971700  20001100  308192000   92448000  
...              ...       ...        ...        ...  
2024-01-24  24867000  26432800  560271000  123369900  
2024-01-25  21021200   9451900  482777000  198076800  
2024-01-26  17803300  12770600  390309000  107343200  
2024-01-29  24510200   6905400  348733000  125013100  
2024-01-30  33477600   6181800  410735000  109982300  

[2536 rows x 50 columns]

In [5]:
# Download data with auto_adjust=True
df = yf.download(tickers, start=start_date, end=end_date, group_by="ticker")

# Reset the multi-level index and convert to long format
df = df.stack(level=0).reset_index()

# Rename columns
df.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Drop 'Adj Close' if it's not needed
df = df.drop(columns=["Adj Close"])

# Removing the time and timezone from the date 
df["Date"] = df["Date"].dt.date 

# Display the first few rows
df.head

[*********************100%***********************]  10 of 10 completed
C:\Users\tanyy\AppData\Local\Temp\ipykernel_31548\3310362398.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=0).reset_index()


<bound method NDFrame.head of              Date Ticker        Open        High         Low       Close  \
0      2014-01-02   AAPL   19.845715   19.893929   19.715000   19.754642   
1      2014-01-02   AMZN   19.940001   19.968000   19.701000   19.898500   
2      2014-01-02    BAC   15.690000   16.160000   15.680000   16.100000   
3      2014-01-02  GOOGL   27.914413   27.971722   27.734234   27.855856   
4      2014-01-02    JPM   58.310001   58.500000   57.970001   58.209999   
...           ...    ...         ...         ...         ...         ...   
25355  2024-01-30   META  403.589996  406.359985  399.570007  400.059998   
25356  2024-01-30   MSFT  412.260010  413.049988  406.450012  408.589996   
25357  2024-01-30   NFLX  567.320007  570.880005  560.820007  562.849976   
25358  2024-01-30   NVDA   62.900002   63.493000   62.259998   62.773998   
25359  2024-01-30   TSLA  195.330002  196.360001  190.610001  191.589996   

          Volume  
0      234684800  
1       42756000  


In [6]:
## INDICATORS

# 50 day moving average
df['50Day_MA'] = df['Close'].transform(lambda x: x.rolling(window=50).mean())

# Support Level (rolling minimum 10 days)
df['Support_Level'] = df.groupby('Ticker')['Low'].transform(lambda x: x.rolling(window=10).min())

# Resistance level (rolling maximum 10 days)
df['Resistance_Level'] = df['High'].transform(lambda x: x.rolling(window=10).max())

# Daily percentage change: percentage change from the previous close to the current close
df['Daily_Percent_Change'] = df.groupby('Ticker')['Close'].pct_change() * 100

# Cumulative Returns: cumulative performance of a stock over time
df['Cumulative_Returns'] = df.groupby('Ticker')['Daily_Percent_Change'].cumsum()

# Roling volume: average volume over a specific period (5 days)
df['Rolling_Volume_5'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=5).mean())

In [7]:
# Load the .env file
load_dotenv()

# Check the environment variables
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')

# Print to verify if they are loaded
if db_host is None or db_user is None:
    print("Environment variables not loaded correctly.")
else:
    print("DB_HOST:", db_host)
    print("DB_USER:", db_user)

DB_HOST: localhost
DB_USER: root


In [ ]:
# Load environment variables
load_dotenv()

# MySQL connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

# Connect to the database
connection = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

#connection.close()